教授にプレゼン後、データの保存をし、テーブルとして値を持つ必要があることがわかった
大きなデータになるので保存方法を考慮する必要があった。
CSV、Excel、バイナリ、HDF5が候補として挙がった。
HDF5形式は大容量のデータセットを効率的に保存するためのファイル形式であるため、採択。
参考サイトの少なさから一時的にSQLを採択
SQLをテストし、全語句のデータの保存をしてみた
SQLでは、VisualStudioCodeの拡張機能が使いづらく、SQLite3に変更した
SQLite->dataframe->変更->dataframe->変更を保存->SQLiteの完成
上記の方法では変更件数に関わらず、全件で1分30秒かかる。
変更に時間がかかりすぎな点が課題であるため、データの更新方法を検討
SQLからPythonに値を渡さず、直接書き込むことで一件当たり0.1秒以内

In [5]:
import pandas as pd
from gensim.models import Word2Vec
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import japanize_matplotlib
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# 学習済みモデルのロード
model_file_path = '../../KITERETU/gw2v160.model'
model = Word2Vec.load(model_file_path)

In [4]:
from gensim.models import Word2Vec
import sqlite3

# データベースに接続し、カーソルを作成する
conn = sqlite3.connect('database.sqlite')
cursor = conn.cursor()

# テーブルが存在しない場合のみ作成する
cursor.execute('''CREATE TABLE IF NOT EXISTS words (word TEXT, value REAL)''')

# モデル内のすべての単語と値をデータベースに保存する
for word in model.wv.key_to_index:
    cursor.execute('''INSERT INTO words VALUES (?, ?)''', (word, 0.000000))

# コミットして変更を確定する
conn.commit()

# データベース接続を閉じる
conn.close()


In [7]:
# SQLiteからデータを取得し、DataFrameに格納する
import pandas as pd

conn = sqlite3.connect('database.sqlite')
df = pd.read_sql_query("SELECT * FROM words", conn)

# キーを使ってデータを変更する
df.loc[df['word'] == '使用', 'value'] = 0.10000

# DataFrameの変更をSQLiteに反映する
conn = sqlite3.connect('database.sqlite')
cursor = conn.cursor()

# データベース内のテーブルを一旦削除する
cursor.execute('''DROP TABLE IF EXISTS words''')

# テーブルを再作成する
cursor.execute('''CREATE TABLE words (word TEXT, value REAL)''')

# DataFrameの内容をデータベースに保存する
for index, row in df.iterrows():
    cursor.execute('''INSERT INTO words VALUES (?, ?)''', (row['word'], row['value']))

# コミットして変更を確定する
conn.commit()

# データベース接続を閉じる
conn.close()

KeyboardInterrupt: 

In [9]:
import sqlite3

# データベースに接続する
conn = sqlite3.connect('database.sqlite')
cursor = conn.cursor()

# データの更新を行う
cursor.execute("UPDATE words SET value = ? WHERE word = ?", (0.10000, '其の'))

# コミットして変更を確定する
conn.commit()

# データベース接続を閉じる
conn.close()


In [ ]:
import sqlite3

# データベースに接続する
conn = sqlite3.connect('database.sqlite')
cursor = conn.cursor()

# データの更新を行う
cursor.execute("UPDATE words SET value = ? WHERE word = ?", (0.10000, '其の'))

# コミットして変更を確定する
conn.commit()

# データベース接続を閉じる
conn.close()
